In [55]:
import pandas as pd
import numpy as np

In [56]:
df=pd.read_csv('titanic.csv')

In [57]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [58]:
df.drop(columns=['PassengerId','Name','Ticket','Cabin'],inplace=True)

In [59]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [60]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.compose import ColumnTransformer

In [61]:
X_train,X_test,y_train,y_test=train_test_split(df.drop(columns=['Survived'],axis=1),df['Survived'],test_size=0.2,random_state=42)

In [62]:
X_train

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
331,1,male,45.5,0,0,28.5000,S
733,2,male,23.0,0,0,13.0000,S
382,3,male,32.0,0,0,7.9250,S
704,3,male,26.0,1,0,7.8542,S
813,3,female,6.0,4,2,31.2750,S
...,...,...,...,...,...,...,...
106,3,female,21.0,0,0,7.6500,S
270,1,male,NaN,0,0,31.0000,S
860,3,male,41.0,2,0,14.1083,S
435,1,female,14.0,1,2,120.0000,S


In [63]:
y_train

331    0
733    0
382    0
704    0
813    0
      ..
106    1
270    0
860    0
435    1
102    0
Name: Survived, Length: 712, dtype: int64

In [64]:
X_train.isnull().sum()

Pclass        0
Sex           0
Age         140
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [65]:
# to fill missing values
trnf1=ColumnTransformer(
    [('simple_imput_age',SimpleImputer(),[2]),
    ('simple_impute_Embarked',SimpleImputer(strategy='most_frequent'),[6])
    ],remainder='passthrough')

In [66]:
# one hot encoding  {drop_first ko true ni kara kuki decision tree use karre h jisme multicollinearity is not an issue}
trnf2=ColumnTransformer(
              [('onehotencoder',OneHotEncoder(sparse=False,handle_unknown='ignore'),[1,6])],remainder='passthrough')

In [67]:
trnf3 = ColumnTransformer([
    ('scale',MinMaxScaler(),slice(0,10))
])

In [68]:
trnf4=DecisionTreeClassifier()

## create pipelines

In [69]:
from sklearn import set_config
set_config(display='diagram')

In [70]:
#using pipeline
pipe=Pipeline([('trnf1',trnf1),
              ('trnf2',trnf2),
              ('trnf3',trnf3),
              ('trnf4',trnf4
              )])

In [71]:
#using make_pipe func
#Pipeline requires naming of steps, make_pipeline does not.
#(Same applies to ColumnTransformer vs make_column_transformer)
pipe=make_pipeline(trnf1,trnf2,trnf3,trnf4)

In [72]:
# train
pipe.fit(X_train,y_train)

Pipeline(steps=[('columntransformer-1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('simple_imput_age',
                                                  SimpleImputer(), [2]),
                                                 ('simple_impute_Embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('columntransformer-2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [1, 6])])),
                ('columntransformer-3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('decisiontreeclassifier', DecisionTreeClassifier())])

In [73]:
# Predict
y_pred = pipe.predict(X_test)

In [74]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.6256983240223464

In [75]:
# cross validation using cross_val_score
from sklearn.model_selection import cross_val_score
cross_val_score(pipe, X_train, y_train, cv=5, scoring='accuracy').mean()


0.6391214419383433

In [78]:

# gridsearchcv
params = {
    'decisiontreeclassifier__max_depth':[1,2,3,4,None]
}

In [79]:

from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(pipe, params, cv=5, scoring='accuracy')
grid.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('columntransformer-1',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('simple_imput_age',
                                                                         SimpleImputer(),
                                                                         [2]),
                                                                        ('simple_impute_Embarked',
                                                                         SimpleImputer(strategy='most_frequent'),
                                                                         [6])])),
                                       ('columntransformer-2',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('onehotencoder',
                                                                         OneHotEncoder(handle_unknown='ignore',
                                                                                       sparse=False),
                                                                         [1,
                                                                          6])])),
                                       ('columntransformer-3',
                                        ColumnTransformer(transformers=[('scale',
                                                                         MinMaxScaler(),
                                                                         slice(0, 10, None))])),
                                       ('decisiontreeclassifier',
                                        DecisionTreeClassifier())]),
             param_grid={'decisiontreeclassifier__max_depth': [1, 2, 3, 4,
                                                               None]},
             scoring='accuracy')

In [80]:
grid.best_score_

0.6391214419383433

In [81]:
grid.best_params_


{'decisiontreeclassifier__max_depth': 2}

In [82]:
# export 
import pickle
pickle.dump(pipe,open('pipe.pkl','wb'))